# Install Packages

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Get the Data Ready

In [ ]:
cont_data_path = "../Data/Contadores.xlsx"
cont = pd.read_excel(cont_data_path)
cont.head()

In [ ]:
print(cont[cont['calibre'] == 0])

As we see, there is an anomalous **calibre** 0. Soon, we will see that these are not present on the **Telemetria** dataset.

In [ ]:
telemetria_data_path = "../Data/TelemetriaConsumosVilaSol_v2.csv"
tel = pd.read_csv(telemetria_data_path)
tel['data'] = pd.to_datetime(tel['data'])
tel.head()

In [ ]:
print(tel.shape)

In [ ]:
print(tel['contact_id'].unique().shape)

In [ ]:
na_count = 0
for i in range(23):
    col_name = f"index_{i}"
    na_count += tel[col_name].isna().sum()

na_count

## Process Telemetry Data

In [ ]:
class Processor():
    def __init__(self, df1, df2):
        self.df1 = df1
        self.df2 = df2
    
    def change_col_names(self):
        '''
        Change column names from index_(num) to just num
        '''
        column_names = self.df1.columns.values # Get the column names
        names_map = {} # Old -> New

        for name in column_names:
            if name.startswith('index'):
                num = name.split('_')[1]
                if len(num) == 1:
                    num = '0' + num # If it is only a single digit
                
                names_map[name] = num

        return self.df1.rename(columns=names_map, inplace=False)

    def add_columns(self, df):
        '''
        Add info from self.df2 to self.df1 (calibre, ano_contador, tipo_consumo)
        '''
        new_df = df.merge(self.df2.drop(columns=["calibre"]), on=['contact_id'], how='left')
        cont = list(set(new_df.loc[new_df["ano_contador"].isna()]["contact_id"]))
        if cont:
            print(f"The following {len(cont)} contadores didn't have matching information available on the other table:")
            print(f"{', '.join(cont)}")
        
        return new_df
    
    def get_differences(self, df):
        """
        Get differences between consecutive values
        """
        hour_cols = [f'{i:02d}' for i in range(24)]

        # Sort by contact_id and date to ensure proper ordering
        df = df.sort_values(['contact_id', 'data']).reset_index(drop=True)

        # Get previous day's information for the same contact_id
        grouped_df = df.groupby(['contact_id', 'id'])

        prev_contact_id = grouped_df['contact_id'].shift(1)
        prev_id = grouped_df['id'].shift(1)
        prev_day = grouped_df['data'].shift(1)
        prev_day_23 = grouped_df['23'].shift(1)

        # Calculate the expected previous date (current date - 1 day)
        expected_prev_date = df['data'] - pd.Timedelta(days=1)
        
        # Only compute if the id right before is the same and only 1 day difference
        valid_prev_day = (df['id'] == prev_id) & (df['contact_id'] == prev_contact_id) & (prev_day == expected_prev_date)
        
        df['consumption_00'] = np.where(valid_prev_day, 
                                        df['00'] - prev_day_23, 
                                        np.nan)

        # Create new columns for hourly consumption (hours 1-23)
        for i in range(1, 24):
            current_hour = hour_cols[i]
            previous_hour = hour_cols[i - 1]
            df[f'consumption_{current_hour}'] = df[current_hour] - df[previous_hour]
        
        
        return df
    
    def remove_duplicate_lc_on_same_date(self, df):
        """
        Identify contact_ids (LCs) that have multiple different ids on the same date.
        """

        # Group by contact_id and data, count unique ids
        lc_date_id_counts = df.groupby(['contact_id', 'data'])['id'].nunique().reset_index()
        lc_date_id_counts.columns = ['contact_id', 'data', 'num_ids']
        
        # Find contact_ids that have more than 1 id on the same date
        problematic_lc_dates = lc_date_id_counts[lc_date_id_counts['num_ids'] > 1]
        
        # Get unique contact_ids that have this problem
        problematic_lcs = problematic_lc_dates['contact_id'].unique()
        
        if len(problematic_lcs) > 0:
            print(f"The following {len(problematic_lcs)} LC had repeated contadores:")
            print(", ".join(problematic_lcs))
            df_cleaned = df[~df['contact_id'].isin(problematic_lcs)].copy()
        else:
            df_cleaned = df.copy()

        
        return df_cleaned


    def correct_cumulative_values(self, df):
        """
        Create cumulative values decreasing
        """
        df_corrected = df.copy()
    
        hour_cols = [f'{i:02d}' for i in range(24)]
        
        # Sort by contact_id, id, and date to ensure proper chronological ordering
        df_corrected = df_corrected.sort_values(['contact_id', 'data']).reset_index(drop=True)
        
        # Track negative consumption occurrences
        negative_records = []
        
        # Process each contact_id and id combination separately
        for (contact_id, meter_id), group in df_corrected.groupby(['contact_id', 'id']):
            indices = group.index.tolist()
            
            # Keep track of the last valid value across dates
            last_value = None
            
            # Process each date sequentially
            for idx in indices:
                # Process each hour sequentially for this date
                for hour in hour_cols:
                    current_value = df_corrected.loc[idx, hour]
                    
                    if pd.isna(current_value):
                        continue

                    # Compare with last_value 
                    if last_value is not None and current_value < last_value:
                        negative_consumption = current_value - last_value
                        negative_records.append(negative_consumption)
                        df_corrected.loc[idx, hour] = last_value
        
                    else:
                        last_value = current_value  # Update to current value
        
        return df_corrected, negative_records


    def process(self):
        '''
        Process everything at once
        '''

        print("Changing column names...")
        df = self.change_col_names()

        print("\nMerging both datasets to get 'tipo_consumo' and 'ano_contador'...")
        merged_df = self.add_columns(df)

        print("\nRemoving LC that have more than 1 contador at the same time")
        merged_df = self.remove_duplicate_lc_on_same_date(merged_df)

        print("\nFixing cumulative values...")
        merged_df, negative_records = self.correct_cumulative_values(merged_df)

        print("\nGetting hourly consumptions...")
        merged_df = self.get_differences(merged_df)
        
        return merged_df, negative_records


proc = Processor(df1=tel, df2=cont)
proc_tel, neg_records = proc.process()

# Save new table
output_path = "../Data/Processed_Telemetria.csv"
proc_tel.to_csv(output_path, index=False, encoding='utf-8')

proc_tel.head()

Let's obtain some statistics regarding the negative consumptions...

In [ ]:
if neg_records:
    negative_series = pd.Series(neg_records)
    
    print("=" * 80)
    print("NEGATIVE CONSUMPTION STATISTICS")
    print("=" * 80)
    print(f"\nTotal negative occurrences: {len(neg_records)}")
    print("\nDescriptive Statistics:")
    print("-" * 80)
    print(negative_series.describe())
    print("=" * 80)
else:
    print("No negative consumption values found!")

In [ ]:
for column in [f'{i:02d}' for i in range(24)]:
    column_name = "consumption_" + column 
    print(f"Number of negatives on column {column_name}: {(proc_tel[column_name] < 0).sum()}")

In [ ]:
print(sorted(list(set(proc_tel['calibre'])))) # Print sorted calibre values

Get all the anomalies that have lower value on hour 23 than on hour 22 (this happens in other hours as well, but in this one is more frequent).

In [15]:
#df = proc_tel.loc[proc_tel['consumption_23'] < 0, ['id', 'contact_id', 'calibre', 'tipo_consumo', 'data', '22', '23']]
#df.to_csv("../Data/Anomalies.csv", index=False)

## Group time series by pair (calibre, tipo_consumo)

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

data_path = "../Data/Processed_Telemetria.csv"
proc_tel = pd.read_csv(data_path)

In [ ]:
proc_tel[proc_tel['contact_id'] == 'LC43022']['calibre'].values

Since 'Serviços 2º contador-Rega Condominio' (20) has **very low** representability, we chose to remove since the analysis would never lead to good results.

In [18]:
# Map pair (calibre, tipo_consumo) to index in future list
df_list = []
pair2idx = {}
idx2pair = []
temp = 0

for idx, ((calibre, consumo), group) in enumerate(proc_tel.groupby(["calibre", "tipo_consumo"])):
    consumo = consumo.strip() # Remove empty space on the right
    if int(calibre) == 20 and consumo == 'Serviços 2º contador-Rega Condominio': # Ignore VERY low representability
        temp = 1
        continue
    
    pair2idx[(int(calibre), consumo)] = idx - temp
    idx2pair.append((int(calibre), consumo))
    df_list.append(group)

In [ ]:
# Test that it works
idx = pair2idx[(20, "Doméstico")]
df_list[idx].head()

In [20]:
# Get the lengths of each subgroup
lista = []

for key in pair2idx.keys():
    idx = pair2idx[key]
    lista.append([len(df_list[idx]), key[0], key[1]]) # (length, calibre, tipo_consumo)    

lista.sort()

In [ ]:
# Function for plotting
def plot(results):

    # Plot
    plt.figure(figsize=(10, 8))
    bars = plt.barh(results["tipo_consumo"] + " (" + results["calibre"].astype(str) + ")", results["length"])
    plt.xlabel("Number of recorded days")
    plt.ylabel("Subgroup - tipo_consumo (calibre)")
    plt.title("Recorded days per subgroup")

    # Add numbers on bars
    for bar, value in zip(bars, results["length"]):
        plt.text(
            value,                             # x-position (at end of bar)
            bar.get_y() + bar.get_height()/2,  # y-position (center of bar)
            str(value),                        # text = length
            va="center", ha="left", fontsize=8 # Small changes to improve
        )

    plt.tight_layout()
    plt.show()

results = pd.DataFrame(lista, columns=["length", "calibre", "tipo_consumo"])
plot(results)

# Data analysis

In [ ]:
def process_and_plot_dataframes(df_list, group_names):
    """
    Process dataframes and create boxplot analyses for all groups in one figure.
    """
    
    n_groups = len(df_list)
    df_list_processed = []
    
    # Create a large figure with subplots: rows = groups, cols = 3 (month, day, hour)
    fig, axes = plt.subplots(n_groups, 3, figsize=(30, 5 * n_groups))
    
    
    fig.suptitle('Consumption Analysis - All Groups', fontsize=16, fontweight='bold', y=0.995)
    
    for idx, (df, (calibre, tipo_consumo)) in enumerate(zip(df_list, group_names)):
        
        # Process the dataframe
        df_processed = process_dataframe(df)
        df_list_processed.append(df_processed)
        
        # Create the three boxplot analyses for this group
        create_boxplots_row(df_processed, calibre, tipo_consumo, axes[idx])
    
    plt.tight_layout(rect=[0, 0, 0.9, 0.99]) # Leave space for title at top
    plt.show()

    return df_list_processed


def process_dataframe(df):
    """
    Transform wide format (date rows, hour columns) to long format with 
    separate month, day, and hour columns.
    """

    # Get Date column name
    date_col = 'data'
    
    # Convert date column to datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Get hour columns 
    hour_cols = [col for col in df.columns if col.isdigit()]
    conshour_cols = [col for col in df.columns if col.startswith('consumption_')]
    
    # Melt the dataframe to long format
    df_long_values = df.melt(
    id_vars=[col for col in df.columns if col not in hour_cols and col not in conshour_cols],
    value_vars=hour_cols,
    var_name='hour',
    value_name='cumulative_value'
    )

    # Melt the consumption columns
    df_long_consumption = df.melt(
        id_vars=[col for col in df.columns if col not in hour_cols and not col.startswith('consumption_')],
        value_vars= conshour_cols,
        var_name='hour',
        value_name='consumption'
    )

    # Extract the hour from consumption column names (remove 'consumption_' prefix)
    df_long_consumption['hour'] = df_long_consumption['hour'].str.replace('consumption_', '')

    # Merge on all id_vars + 'hour'
    id_cols = [col for col in df.columns if col not in hour_cols and col not in conshour_cols]
    df_long = df_long_values.merge(
        df_long_consumption,
        on=id_cols + ['hour']
    )
    
    # Extract month, day of week, and convert hour to integer
    df_long['month'] = df_long[date_col].dt.month
    df_long['month_name'] = df_long[date_col].dt.month_name().apply(lambda x: x[:3])
    df_long['day_of_week'] = df_long[date_col].dt.dayofweek
    df_long['day_name'] = df_long[date_col].dt.day_name().apply(lambda x: x[:3])
    df_long['hour'] = df_long['hour'].astype(int)
    
    return df_long


def create_boxplots_row(df, calibre, tipo_consumo, axes_row):
    """
    Create three boxplots in a single row for a processed dataframe.
    """

    x_fontsize = y_fontsize = 15
    title_fontsize = 17

    # Remove any NaN values
    df = df.dropna(subset=['consumption'])

    # Add row label
    axes_row[0].text(-0.15, 0.5, f'Calibre: {calibre}\nTipo: {tipo_consumo}', 
                     transform=axes_row[0].transAxes, fontsize=20, 
                     fontweight='bold', va='center', ha='right')
    
    # Boxplot by Month
    month_order = sorted(df['month'].unique())
    month_labels = [df[df['month'] == m]['month_name'].iloc[0] for m in month_order]
    data_by_month = [df[df['month'] == m]['consumption'].values for m in month_order]
    
    axes_row[0].boxplot(data_by_month, tick_labels=month_labels)
    axes_row[0].set_xlabel('Month', fontweight='bold', fontsize=x_fontsize)
    axes_row[0].set_ylabel('Consumption Value', fontweight='bold', fontsize=y_fontsize)
    axes_row[0].set_title('By Month', fontsize=title_fontsize)
    axes_row[0].tick_params(axis='x', rotation=45, labelsize=11)
    axes_row[0].tick_params(axis='y', labelsize=11)
    axes_row[0].grid(True, alpha=0.3)
    
    # Boxplot by Day of Week
    day_order = list(range(7))  # 0=Monday, 6=Sunday
    day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    data_by_day = [df[df['day_of_week'] == d]['consumption'].values 
                   for d in day_order if d in df['day_of_week'].values]
    valid_day_labels = [day_labels[d] for d in day_order if d in df['day_of_week'].values]
    
    axes_row[1].boxplot(data_by_day, tick_labels=valid_day_labels)
    axes_row[1].set_xlabel('Day of Week', fontweight='bold', fontsize=x_fontsize)
    axes_row[1].set_ylabel('Consumption Value', fontweight='bold', fontsize=y_fontsize)
    axes_row[1].set_title('By Day of Week', fontsize=title_fontsize)
    axes_row[1].tick_params(axis='x', labelsize=11)
    axes_row[1].tick_params(axis='y', labelsize=11)
    axes_row[1].grid(True, alpha=0.3)
    
    # Boxplot by Hour
    hour_order = sorted(df['hour'].unique())
    data_by_hour = [df[df['hour'] == h]['consumption'].values for h in hour_order]
    
    axes_row[2].boxplot(data_by_hour, tick_labels=[f'{h:02d}' for h in hour_order])
    axes_row[2].set_xlabel('Hour', fontweight='bold', fontsize=x_fontsize)
    axes_row[2].set_ylabel('Consumption Value', fontweight='bold', fontsize=y_fontsize)
    axes_row[2].set_title('By Hour', fontsize=title_fontsize)
    axes_row[2].tick_params(axis='x', rotation=45, labelsize=11)
    axes_row[2].tick_params(axis='y', labelsize=11)
    axes_row[2].grid(True, alpha=0.3)


df_processed = process_and_plot_dataframes(df_list=df_list, group_names=idx2pair)

In [23]:
import numpy as np
import json
from scipy import stats
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram, cophenet
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


class SubgroupIdentifier:
    """
    Identify subgroups within time series data based on statistical similarity.
    Uses distribution comparison methods to cluster similar time periods.
    """
    
    def __init__(self, df, calibre, tipo_consumo):
        self.df = df.dropna(subset=['consumption'])
        self.calibre = calibre
        self.tipo_consumo = tipo_consumo
        
    def compare_distributions(self, group1_data, group2_data, method='ks'):
        """
        Compare two distributions using statistical tests.
        """
        if len(group1_data) == 0 or len(group2_data) == 0:
            return 1.0
        
        if method == 'ks':
            statistic, _ = stats.ks_2samp(group1_data, group2_data)
            return statistic
            
                
        elif method == 'wasserstein':
            dist = stats.wasserstein_distance(group1_data, group2_data)
            data_range = max(group1_data.max(), group2_data.max()) - \
                        min(group1_data.min(), group2_data.min())
            return dist / data_range if data_range > 0 else 0
        
        elif method == 'kl':
            # Determine shared bin edges
            min_val = min(group1_data.min(), group2_data.min())
            max_val = max(group1_data.max(), group2_data.max())
            
            # Sturges Rule
            n_bins = int(np.ceil(np.log2(len(group1_data) + len(group2_data)) + 1))
            bins = np.linspace(min_val, max_val, n_bins + 1)
            
            # Create histograms
            hist1, _ = np.histogram(group1_data, bins=bins)
            hist2, _ = np.histogram(group2_data, bins=bins)
            
            # Convert to probability distributions 
            epsilon = 1e-10
            p = (hist1 + epsilon) / (hist1.sum() + epsilon * n_bins)
            q = (hist2 + epsilon) / (hist2.sum() + epsilon * n_bins)
            
            # Compute symmetric KL divergence
            kl_pq = stats.entropy(p, q)
            kl_qp = stats.entropy(q, p)
            
            return (kl_pq + kl_qp) / 2 # No correction needed for sufficiently small bin wideness
    
    def _compute_dunn_index(self, distance_matrix, labels):
        """
        Compute Dunn Index (0, 1)
        """
        n_clusters = len(np.unique(labels))
        if n_clusters <= 1:
            return 0
        
        # Compute inter-cluster distances (minimum distance between clusters)
        min_inter_cluster = float('inf')
        for i in range(n_clusters):
            for j in range(i + 1, n_clusters):
                cluster_i_indices = np.where(labels == i)[0]
                cluster_j_indices = np.where(labels == j)[0]
                
                inter_dists = distance_matrix[np.ix_(cluster_i_indices, cluster_j_indices)] # Get every pairwise distance between points of cluster i and j
                min_inter_cluster = min(min_inter_cluster, np.min(inter_dists))
        
        # Compute intra-cluster distances (maximum diameter within clusters)
        max_intra_cluster = 0
        for i in range(n_clusters):
            cluster_indices = np.where(labels == i)[0]
            if len(cluster_indices) > 1:
                intra_dists = distance_matrix[np.ix_(cluster_indices, cluster_indices)]
                max_intra_cluster = max(max_intra_cluster, np.max(intra_dists))
        
        if max_intra_cluster == 0:
            return 0
        
        return min_inter_cluster / max_intra_cluster
    
    def _compute_c_index(self, distance_matrix, labels):
        """
        Compute C-Index (0, 1)
        """
        n = len(labels)
        n_clusters = len(np.unique(labels))
        
        if n_clusters <= 1 or n <= 1:
            return 1
        
        # Get all pairwise distances
        condensed_dist = squareform(distance_matrix)
        all_distances = np.sort(condensed_dist)
        
        # Compute within-cluster distances
        within_cluster_dists = []
        for i in range(n):
            for j in range(i + 1, n):
                if labels[i] == labels[j]:
                    within_cluster_dists.append(distance_matrix[i, j])
        
        if len(within_cluster_dists) == 0:
            return 1
        
        S_w = np.sum(within_cluster_dists)
        n_w = len(within_cluster_dists)
        
        # Min and max possible sums
        S_min = np.sum(all_distances[:n_w])
        S_max = np.sum(all_distances[-n_w:])
        
        if S_max - S_min == 0:
            return 0
        
        return (S_w - S_min) / (S_max - S_min)
    
    def _compute_silhouette(self, distance_matrix, labels):
        """
        Compute Silhouette Coefficient (-1, 1)
        """
        n_clusters = len(np.unique(labels))
        if n_clusters <= 1 or n_clusters >= len(labels):
            return -1
        
        try:
            return silhouette_score(distance_matrix, labels, metric='precomputed')
        except:
            return -1
    
    def _evaluate_clustering(self, distance_matrix, labels):
        """
        Evaluate clustering quality using multiple metrics.
        Returns dict with all metrics.
        """
        return {
            'dunn': self._compute_dunn_index(distance_matrix, labels),
            'c_index': self._compute_c_index(distance_matrix, labels),
            'silhouette': self._compute_silhouette(distance_matrix, labels)
        }
    
    def _find_optimal_clusters(self, linkage_matrix, max_clusters=5):
        """
        Find optimal number of clusters using elbow method on dendrogram heights.
        """
        if len(linkage_matrix) < 2:
            return 1
        
        heights = linkage_matrix[:, 2]
        if len(heights) < 3:
            return 2
        
        diffs = np.diff(heights)
        if len(diffs) > 0:
            largest_jump_idx = np.argmax(diffs)
            optimal_clusters = len(heights) - largest_jump_idx
            return min(max(2, optimal_clusters), max_clusters)
        
        return 2
    
    def find_best_clustering_method(self, period_type='month', methods=['ks', 'wasserstein', 'kl'],
                                   linkage_methods=['average', 'complete', 'single']):
        """
        Test all combinations of distance metrics and linkage methods.
        Select best based on validation indices ranking.
        """
        col_map = {
            'month': 'month_name',
            'day': 'day_name',
            'hour': 'hour'
        }
        col = col_map[period_type]
        
        periods = sorted(self.df[col].unique())
        n_periods = len(periods)
        
        if n_periods <= 1:
            return None, {'groups': [periods], 'dendrogram_data': None, 'distance_matrix': None}
        
        results = []
        
        for method in methods:
            # Create distance matrix
            distance_matrix = np.zeros((n_periods, n_periods))
            
            for i, period1 in enumerate(periods):
                for j, period2 in enumerate(periods):
                    if i < j:
                        data1 = self.df[self.df[col] == period1]['consumption'].values
                        data2 = self.df[self.df[col] == period2]['consumption'].values
                        
                        dist = self.compare_distributions(data1, data2, method=method)
                        if dist < 0:
                            print(method)

                        distance_matrix[i, j] = dist
                        distance_matrix[j, i] = dist
            
            condensed_dist = squareform(distance_matrix)
            
            # Find best linkage method using cophenetic correlation
            best_linkage_method = None
            best_coph_corr = -1
            best_linkage_matrix = None
            
            for link_method in linkage_methods: 
                try:                  
                    link_matrix = linkage(condensed_dist, method=link_method)
                    coph_corr, _ = cophenet(link_matrix, condensed_dist) # Needs new and old distances
                    
                    if coph_corr > best_coph_corr:
                        best_coph_corr = coph_corr
                        best_linkage_method = link_method
                        best_linkage_matrix = link_matrix
                except:
                    continue
            
            if best_linkage_matrix is None:
                continue

            # Find optimal number of clusters
            n_clusters = self._find_optimal_clusters(best_linkage_matrix, 
                                                     max_clusters=min(5, n_periods))
            
            # Get cluster labels
            clustering = AgglomerativeClustering(n_clusters=n_clusters,
                                                linkage=best_linkage_method,
                                                metric='precomputed')
            
            labels = clustering.fit_predict(distance_matrix)
            
            # Evaluate clustering
            metrics = self._evaluate_clustering(distance_matrix, labels)
            
            results.append({
                'method': method,
                'linkage': best_linkage_method,
                'n_clusters': n_clusters,
                'coph_corr': best_coph_corr,
                'dunn': metrics['dunn'],
                'c_index': metrics['c_index'],
                'silhouette': metrics['silhouette'],
                'distance_matrix': distance_matrix,
                'linkage_matrix': best_linkage_matrix,
                'labels': labels
            })
        
        if not results:
            return None, {'groups': [periods], 'dendrogram_data': None, 'distance_matrix': None}
        
        # Rank each result by each metric
        # Dunn: higher is better
        # C-index: lower is better
        # Silhouette: higher is better
        dunn_sorted_indices = np.argsort([-r['dunn'] for r in results])
        c_index_sorted_indices = np.argsort([r['c_index'] for r in results])
        silhouette_sorted_indices = np.argsort([-r['silhouette'] for r in results])

        # Convert sorted indices to ranks
        dunn_ranks = np.empty(len(results), dtype=int)
        dunn_ranks[dunn_sorted_indices] = np.arange(1, len(results) + 1)

        c_index_ranks = np.empty(len(results), dtype=int)
        c_index_ranks[c_index_sorted_indices] = np.arange(1, len(results) + 1)

        silhouette_ranks = np.empty(len(results), dtype=int)
        silhouette_ranks[silhouette_sorted_indices] = np.arange(1, len(results) + 1)
        
        # Compute mean rank
        for i, result in enumerate(results):
            result['mean_rank'] = np.mean([dunn_ranks[i], c_index_ranks[i], silhouette_ranks[i]])
        
        # Sort by mean rank, then by number of clusters
        results.sort(key=lambda x: (x['mean_rank'], x['n_clusters']))
        
        best = results[0]
        
        # Group periods by cluster
        groups = [[] for _ in range(best['n_clusters'])]
        for period, label in zip(periods, best['labels']):
            groups[label].append(str(period))
        
        groups = [sorted(g) for g in groups if g]
        
        result_dict = {
            'groups': groups,
            'dendrogram_data': best['linkage_matrix'],
            'distance_matrix': best['distance_matrix'],
            'period_labels': periods,
            'method': best['method'],
            'linkage': best['linkage'],
            'n_clusters': best['n_clusters'],
            'metrics': {
                'dunn': round(best['dunn'], 3),
                'c_index': round(best['c_index'], 3),
                'silhouette': round(best['silhouette'], 3),
                'cophenetic': round(best['coph_corr'], 3)
            }
        }
        
        return result_dict
    
    def plot_analysis(self, result, period_type='month'):
        """
        Create visualization of clustering analysis with distribution boxplots.
        """
        if result['dendrogram_data'] is None:
            print(f"Not enough {period_type}s to cluster")
            return
        
        fig, axes = plt.subplots(1, 3, figsize=(24, 6))
        
        n_groups = len(result['groups'])
        
        # Prepare data for boxplot
        match period_type:
            case 'month':
                order = sorted(self.df['month'].unique())
                labels = [self.df[self.df['month'] == m]['month_name'].iloc[0] for m in order]
                data = [self.df[self.df['month'] == m]['consumption'].values for m in order]
            case 'day':
                day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
                order = [d for d in range(7) if d in self.df['day_of_week'].values]
                labels = [day_labels[d] for d in order]
                data = [self.df[self.df['day_of_week'] == d]['consumption'].values for d in order]
            case 'hour':
                order = sorted(self.df['hour'].unique())
                labels = [f'{h:02d}' for h in order]
                data = [self.df[self.df['hour'] == h]['consumption'].values for h in order]
        
        # Plot boxplot
        axes[0].boxplot(data, tick_labels=labels, patch_artist=True)
        axes[0].set_xlabel(period_type.capitalize(), fontweight='bold', fontsize=13)
        axes[0].set_ylabel('Consumption Value', fontweight='bold', fontsize=13)
        axes[0].set_title(f'Distribution by {period_type.capitalize()}', fontsize=15)
        axes[0].tick_params(axis='x', rotation=45, labelsize=11)
        axes[0].tick_params(axis='y', labelsize=11)
        axes[0].grid(True, alpha=0.3)
        
        # Plot dendrogram
        threshold = result['dendrogram_data'][-n_groups, 2] + 1e-10
        dendrogram(result['dendrogram_data'], 
                  labels=result['period_labels'],
                  ax=axes[1],
                  color_threshold=threshold)
        
        axes[1].set_title(f'Hierarchical Clustering - {period_type.capitalize()}s', fontsize=15)
        axes[1].set_xlabel(period_type.capitalize(), fontweight='bold', fontsize=13)
        axes[1].set_ylabel('Distance', fontweight='bold', fontsize=13)
        axes[1].tick_params(axis='x', rotation=45, labelsize=11)
        axes[1].tick_params(axis='y', labelsize=11)
        
        # Plot distance matrix heatmap
        im = axes[2].imshow(result['distance_matrix'], cmap='YlOrRd', aspect='auto')
        axes[2].set_xticks(range(len(result['period_labels'])))
        axes[2].set_yticks(range(len(result['period_labels'])))
        axes[2].set_xticklabels(result['period_labels'], rotation=45, fontsize=11)
        axes[2].set_yticklabels(result['period_labels'], fontsize=11)
        axes[2].set_title(f'Distance Matrix - {period_type.capitalize()}s', fontsize=15)
        plt.colorbar(im, ax=axes[2], label='Distance')
        
        # Add method info to title
        method_info = f"{result['method'].upper()}, {result['linkage']}, k={result['n_clusters']}"
        fig.suptitle(f'Clustering Analysis - {period_type.capitalize()} | {self.tipo_consumo} ({self.calibre})\n{method_info}', 
                    fontsize=18, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\nBest configuration: {result['method']} + {result['linkage']} (k={result['n_clusters']})")
        print(f"Metrics: Dunn={result['metrics']['dunn']:.3f}, C-index={result['metrics']['c_index']:.3f}, "
              f"Silhouette={result['metrics']['silhouette']:.3f}, Cophenetic={result['metrics']['cophenetic']:.3f}")
        print(f"\nIdentified {n_groups} subgroups:")
        for i, group in enumerate(result['groups'], 1):
            print(f"  Group {i}: {group}")
        
        return result
    
    def generate_json_config(self, day_result, month_result, hour_result):
        """
        Generate complete JSON configuration for this group.
        """
        hour_groups = []
        for group in hour_result['groups']:
            formatted_group = [f"{int(h):02d}" for h in group]
            hour_groups.append(formatted_group)
        
        config = {
            "calibre": str(self.calibre),
            "tipo_consumo": self.tipo_consumo,
            "month_groups": month_result['groups'],
            "day_groups": day_result['groups'],
            "hour_groups": hour_groups,
            "methods": {
                "month": {
                    "metric": month_result['method'],
                    "linkage": month_result['linkage'],
                    "n_clusters": int(month_result['n_clusters']),
                    "quality": month_result['metrics']
                },
                "day": {
                    "metric": day_result['method'],
                    "linkage": day_result['linkage'],
                    "n_clusters": int(day_result['n_clusters']),
                    "quality": day_result['metrics']
                },
                "hour": {
                    "metric": hour_result['method'],
                    "linkage": hour_result['linkage'],
                    "n_clusters": int(hour_result['n_clusters']),
                    "quality": hour_result['metrics']
                }
            }
        }
        
        return config


def analyze_all_groups(df_list, idx2pair, output_path='../info/subgroups.json'):
    """
    Analyze all groups with automatic method selection and generate combined JSON configuration.
    """
    all_configs = []
    
    for idx, df in enumerate(df_list):
        calibre, tipo_consumo = idx2pair[idx]
        print(f"\n{'='*60}")
        print(f"Analyzing: {tipo_consumo} ({calibre})")
        print('='*60)
        
        identifier = SubgroupIdentifier(df, calibre, tipo_consumo)
        
        # Find best methods automatically
        print("\n--- MONTHS ---")
        month_result = identifier.find_best_clustering_method('month')
        identifier.plot_analysis(month_result, 'month')
        
        print("\n--- DAYS ---")
        day_result = identifier.find_best_clustering_method('day')
        identifier.plot_analysis(day_result, 'day')
        
        print("\n--- HOURS ---")
        hour_result = identifier.find_best_clustering_method('hour')
        identifier.plot_analysis(hour_result, 'hour')
        
        # Generate config
        config = identifier.generate_json_config(day_result, month_result, hour_result)
        all_configs.append(config)
    
    # Save all configs
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump({"groups": all_configs}, f, indent=2, ensure_ascii=False)
    
    print(f"\nConfiguration saved to {output_path}")
    return all_configs

In [ ]:
analyze_all_groups(df_processed, idx2pair)

In [ ]:
idx = pair2idx[(15, 'Doméstico')]
df_processed[idx].head()

In [27]:
seq_all_df = pd.concat(df_processed)
output_path = '../Data/input_ts.csv'
seq_all_df.to_csv(output_path)

# Divide into groups

In [ ]:
from itertools import product

def load_subgroups(json_path):
    """Load the subgroups JSON file."""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    return data['groups']

def divide_dataframes(df_list, idx2pair, json_path):
    """
    Divide each DataFrame in df_list into subgroups based on the JSON configuration.
    
    Returns:
    --------
    result : List with strcuture:
                'df': pd.DataFrame,
                'months': list,
                'days': list,
                'hours': list
                
    """
    # Load subgroup configurations
    groups_config = load_subgroups(json_path)
    
    # Create a lookup dictionary for faster access
    config_lookup = {}
    for config in groups_config:
        key = (int(config['calibre']), config['tipo_consumo'])
        config_lookup[key] = config
    

    # Result 
    result = []
    
    # Process each DataFrame
    for idx, df in enumerate(df_list):
        # Get the (calibre, tipo_consumo) pair for this DataFrame
        pair = idx2pair[idx]

        # Get the configuration for this pair
        if pair not in config_lookup:
            print(f"Warning: No configuration found for {pair}, skipping index {idx}")
            continue
        
        config = config_lookup[pair]
        
        # Get all subgroup combinations
        month_groups = config['month_groups']
        day_groups = config['day_groups']
        hour_groups = config['hour_groups']
        
        # Create all combinations
        subgroups = []
        for months, days, hours in product(month_groups, day_groups, hour_groups): # Cartesian product
            hours = list(map(int, hours))
            
            # Filter the DataFrame
            mask = (
                (df['month_name'].isin(months)) &
                (df['day_name'].isin(days)) &
                (df['hour'].isin(hours))
            )
            
            subgroup_df = df[mask].copy()
            
            # Only add non-empty subgroups
            if len(subgroup_df) > 0:
                subgroups.append({
                    'df': subgroup_df,
                    'months': months,
                    'days': days,
                    'hours': hours
                })
        
        result.append(subgroups)
    
    return result

subgroups = divide_dataframes(df_processed, idx2pair, '../info/subgroups.json')

# Access subgroups for a specific original DataFrame index
for i, subgroup in enumerate(subgroups[0]):
    print(f"Subgroup {idx2pair[0]}:")
    print(f"  Months: {subgroup['months']}")
    print(f"  Days: {subgroup['days']}")
    print(f"  Hours: {sorted(subgroup['hours'])}")
    print(f"  DataFrame shape: {subgroup['df'].shape}")
    print()

# Export Data

In [30]:
import os

# Create folder "variables" if it doesn't exist
os.makedirs("../variables", exist_ok=True)

In [ ]:
pair = (40, 'Comércio')
idx = pair2idx[pair]
for subgroup in subgroups[idx]:
    print(f"  Months: {subgroup['months']}")
    print(f"  Days: {subgroup['days']}")
    print(f"  Hours: {sorted(subgroup['hours'])}")
    print(f"  DataFrame shape: {subgroup['df'].shape}")
    print()

In [32]:
import pickle

# Combine everything into one object
data = {
    "idx2pair": idx2pair,
    "pair2idx": pair2idx,
    "subgroups": subgroups,
}

# Save all as a single pickle file
with open("../variables/variables.pkl", "wb") as f:
    pickle.dump(data, f)